In [38]:
from nbed.driver import NbedDriver
from nbed.ham_converter import HamiltonianConverter
from openfermion import get_sparse_operator
import scipy as sp
from openfermion.utils import count_qubits
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np
import os
from nbed.mol_plot import Draw_molecule
from nbed.utils import pubchem_mol_geometry, build_ordered_xyz_string

In [39]:
H2O_struct_dict = pubchem_mol_geometry('H2O')

H2O_struct_dict

{0: ('O', (0.0, 0.0, 0.0)),
 1: ('H', (0.2774, 0.8929, 0.2544)),
 2: ('H', (0.6068, -0.2383, -0.7169))}

In [40]:
H2O_xyz = build_ordered_xyz_string(H2O_struct_dict, [0,1,2])
print(H2O_xyz)

3
 
O	0.0	0.0	0.0
H	0.2774	0.8929	0.2544
H	0.6068	-0.2383	-0.7169



In [68]:
# https://github.com/nutjunkie/IQmol/blob/master/share/fragments/Molecules/Water.xyz
H2O_xyz = f'''3

O    0.0000000    0.0000000   {-0.1653507+0.1653507}
H    0.7493682    0.0000000    {0.4424329+0.1653507}
H   -0.7493682    0.0000000    {0.4424329+0.1653507}'''
print(H2O_xyz)

3

O    0.0000000    0.0000000   0.0
H    0.7493682    0.0000000    0.6077836
H   -0.7493682    0.0000000    0.6077836


In [93]:
structural_dict= {}
for ind, atom_xyz in enumerate(H2O_xyz[3:].split('\n')):
    atom_xyz_list = atom_xyz.split()
    structural_dict[ind] = [atom_xyz_list[0], tuple(map(float, (atom_xyz_list[i] for i in range(1,4)))) ]

structural_dict

{0: ['O', (0.0, 0.0, 0.0)],
 1: ['H', (0.7493682, 0.0, 0.6077836)],
 2: ['H', (-0.7493682, 0.0, 0.6077836)]}

In [96]:
theta = np.arctan(structural_dict[1][1][0] / structural_dict[1][1][2])

list_bond_lengths = [0.5, 0.75, 1, 1.25, 1.5, 2, 3] 

new_x_vals = [H_length*np.cos(theta) for H_length in list_bond_lengths]
new_z_vals = [H_length*np.sin(theta) for H_length in list_bond_lengths]

In [97]:
xyz_structures = []

for new_x, new_z in zip(new_x_vals, new_z_vals):
    dict_temp = structural_dict.copy()
    
    dict_temp[1][1] = new_x
    dict_temp[1][3] = new_z
    
    temp_xyz = build_ordered_xyz_string(dict_temp, [0,1,2])
    xyz_structures.append(temp_xyz)


IndexError: list assignment index out of range

In [81]:
mol_viewer = Draw_molecule(xyz_structures[2], style='stick')
mol_viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [77]:
print(xyz_structures[0])

3

    O    0.0000000    0.0000000   0.6077836
    H    0.3149597    0.0000000    0.3883303
    H   -0.7493682    0.0000000    0.6077836


In [34]:
H2O_xyz_array = np.array([
    [0.0 ,0.0 ,0.0],
    [0.7493682, 0.0000000, 0.6077836],
    [-0.7493682, 0.0000000, 0.6077836]
])

In [35]:
Ry = lambda theta: np.array([
    [np.cos(theta) , 0 , np.sin(theta)],
    [0,              1 ,     0],
    [np.sin(theta), 0,  np.cos(theta)]
])

In [37]:
Ry(np.deg2rad(12))@H2O_xyz_array[2,:]

array([-0.60662739,  0.        ,  0.43869966])

In [30]:
mol_viewer = Draw_molecule(H2O_xyz, style='stick')
mol_viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [14]:
localization = 'spade'
n_active_atoms = 1
geometry = H2O_xyz

In [15]:
basis = 'STO-3G'
xc_functional = 'B3LYP'
projector = 'both'
convergence = 1e-6
charge = 0
mu_level_shift=1e6
run_ccsd_emb = True
run_fci_emb = True
max_ram_memory = 8_000
pyscf_print_level = 1
unit = 'angstrom'
occupied_threshold = 0.95
virtual_threshold = 0.95

In [17]:
driver = NbedDriver(
        geometry=geometry,
        n_active_atoms=n_active_atoms,
        basis=basis,
        xc_functional=xc_functional,
        projector=projector,
        localization=localization,
        convergence=convergence,
        charge=charge,
        mu_level_shift=mu_level_shift,
        run_ccsd_emb=run_ccsd_emb,
        run_fci_emb=run_fci_emb,
        max_ram_memory=max_ram_memory,
        pyscf_print_level=pyscf_print_level,
        unit=unit,
        occupied_threshold=occupied_threshold,
        virtual_threshold=virtual_threshold,
    )



In [23]:
## global
global_dft = driver._global_rks.e_tot
# global_fci = driver._global_fci.e_tot
global_H = driver.full_system_hamiltonian
# global_H_sparse = get_sparse_operator(global_H)
# E_fci =  sp.sparse.linalg.eigsh(global_H_sparse, k=1, which='SA')[0][0] + driver._global_hf.energy_nuc()
global_qubit_Ham = getattr( HamiltonianConverter(global_H, transform='jordan_wigner'), 'openfermion')
global_nqubits = count_qubits(global_qubit_Ham)

# HUZ
# huz_RHF = driver._huzinaga['e_rhf']
# huz_ccsd = driver._huzinaga['e_ccsd']
huz_fci = driver._huzinaga['e_fci']
huz_H_emb = driver._huzinaga['hamiltonian']
# huz_classical = driver._huzinaga['classical_energy']
# huz_H_sparse = get_sparse_operator(huz_H_emb)
# huz_e_emb = sp.sparse.linalg.eigsh(huz_H_sparse, k=1, which='SA')[0][0]
huz_qubit_Ham = getattr( HamiltonianConverter(huz_H_emb, transform='jordan_wigner'), 'openfermion')
huz_nqubits = count_qubits(huz_qubit_Ham)

# mu shift
# mu_RHF = driver._mu['e_rhf']
# mu_ccsd = driver._mu['e_ccsd']
mu_fci = driver._mu['e_fci']
mu_H_emb = driver._mu['hamiltonian']
# mu_classical = driver._mu['classical_energy']
# mu_H_sparse = get_sparse_operator(mu_H_emb)
# mu_e_emb = sp.sparse.linalg.eigsh(mu_H_sparse, k=1, which='SA')[0][0]
mu_qubit_Ham = getattr( HamiltonianConverter(mu_H_emb, transform='jordan_wigner'), 'openfermion')
mu_nqubits = count_qubits(mu_qubit_Ham)

In [19]:
huz_nqubits

10

In [20]:
huz_H_sparse = get_sparse_operator(huz_H_emb)
huz_e_emb = sp.sparse.linalg.eigsh(huz_H_sparse, k=1, which='SA')[0][0]

mu_H_sparse = get_sparse_operator(mu_H_emb)
mu_e_emb = sp.sparse.linalg.eigsh(mu_H_sparse, k=1, which='SA')[0][0]

In [22]:
huz_e_emb - mu_e_emb

6.660717701834074e-07

In [25]:
huz_fci - huz_e_emb

8.526512829121202e-14

In [26]:
mu_fci - mu_e_emb

8.668621376273222e-13

In [ ]:
# global_dft_list=[]
# global_fci_list=[]
# global_H_diag_list=[]
# global_qubit_H_length_list=[]
# global_qubit_H_nqubits=[]

# huz_rhf_list=[]
# huz_ccsd_list=[]
# huz_fci_list=[]
# huz_H_diag_list=[]
# huz_qubit_H_length_list=[]
# huz_H_nqubitsa=[]

# mu_rhf_list=[]
# mu_ccsd_list=[]
# mu_fci_list=[]
# mu_H_diag_list=[]
# mu_qubit_H_length_list=[]
# mu_H_nqubits=[]
# N_active_AOs = []

# for geometry_path in tqdm(bottom_top_bottom_top_file_paths):
#     try:
#         driver = NbedDriver(
#                 geometry=geometry_path,
#                 n_active_atoms=n_active_atoms,
#                 basis=basis,
#                 xc_functional=xc_functional,
#                 projector=projector,
#                 localization=localization,
#                 convergence=convergence,
#                 charge=charge,
#                 mu_level_shift=mu_level_shift,
#                 run_ccsd_emb=run_ccsd_emb,
#                 run_fci_emb=run_fci_emb,
#                 max_ram_memory=max_ram_memory,
#                 pyscf_print_level=pyscf_print_level,
#                 unit=unit,
#                 occupied_threshold=occupied_threshold,
#                 virtual_threshold=virtual_threshold)
#     except:
#         print('error for:')
#         print(geometry_path)
#         print()
#         continue

    
#     ## global
#     global_dft_list.append(driver._global_rks.e_tot)
#     global_fci_list.append(driver._global_fci.e_tot)
    
#     global_H = driver.full_system_hamiltonian
#     global_H_sparse = get_sparse_operator(global_H)
#     E_fci =  sp.sparse.linalg.eigsh(global_H_sparse, k=1, which='SA')[0][0] + driver._global_hf.energy_nuc()
#     global_H_diag_list.append(E_fci)
    
#     global_qubit_Ham = getattr( HamiltonianConverter(global_H, transform='jordan_wigner'), 'openfermion')
#     global_qubit_H_length_list.append(len(global_qubit_Ham.terms))
#     global_qubit_H_nqubits.append(count_qubits(global_qubit_Ham))

#     ##
#     N_active_AOs.append(len(driver.localized_system.active_MO_inds))
    
#     # HUZ
#     huz_rhf_list.append(driver._huzinaga['e_rhf'])
#     huz_ccsd_list.append(driver._huzinaga['e_ccsd'])
#     huz_fci_list.append(driver._huzinaga['e_fci'])

#     huz_H_emb = driver._huzinaga['hamiltonian']
#     huz_classical = driver._huzinaga['classical_energy']
#     huz_H_sparse = get_sparse_operator(huz_H_emb)
#     huz_e_emb = sp.sparse.linalg.eigsh(huz_H_sparse, k=1, which='SA')[0][0]
#     huz_H_diag_list.append(huz_e_emb)
    
#     huz_qubit_Ham = getattr( HamiltonianConverter(huz_H_emb, transform='jordan_wigner'), 'openfermion')
#     huz_qubit_H_length_list.append(len(huz_qubit_Ham.terms))
#     huz_nqubits = count_qubits(huz_qubit_Ham)
#     huz_H_nqubits.append(huz_nqubits)

#     # mu shift
#     mu_rhf_list.append(driver._mu['e_rhf'])
#     mu_ccsd_list.append(driver._mu['e_ccsd'])
#     mu_fci_list.append(driver._mu['e_fci'])
    
#     mu_H_emb = driver._mu['hamiltonian']
#     mu_classical = driver._mu['classical_energy']
#     mu_H_sparse = get_sparse_operator(mu_H_emb)
#     mu_e_emb = sp.sparse.linalg.eigsh(mu_H_sparse, k=1, which='SA')[0][0]
#     mu_H_diag_list.append(mu_e_emb)
    
#     mu_qubit_Ham = getattr( HamiltonianConverter(mu_H_emb, transform='jordan_wigner'), 'openfermion')
#     mu_nqubits = count_qubits(huz_qubit_Ham)
#     mu_qubit_H_length_list.append(len(mu_qubit_Ham.terms))
#     mu_H_nqubits.append(mu_nqubits)
    
#     del driver

In [ ]:
import pandas as pd
exp_data = {
    'global DFT': global_dft_list, 
    
    'global FCI': global_fci_list,
    'huz emb FCI': huz_H_diag_list,
    'mu emb FCI': mu_H_diag_list,
    
    'global H': global_qubit_H_length_list,
    'huz H emb': huz_qubit_H_length_list,
    'mu H emb': mu_qubit_H_length_list,
    
    'global n qubits ': global_qubit_H_nqubits,
    'huz n qubits ': huz_H_nqubits,
    'mu n qubits ': mu_H_nqubits,
    'N active AOs ': N_active_AOs,
    
}

df = pd.DataFrame(data=exp_data)

df

In [ ]:
%matplotlib inline

angles = list(range(85,96))
# angles = list(range(85,87))

plt.plot(angles, global_dft_list, label='global DFT', color='b')
plt.plot(angles, global_fci_list, label='global FCI', color='orange')
plt.plot(angles, huz_H_diag_list, label='huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(angles, mu_H_diag_list, label='mu emb', linestyle='-', marker='x', color='r')

plt.xlabel('angle (degrees)')
plt.ylabel('energy (Ha)')
plt.grid(True)
# plt.savefig("test.png")
plt.legend()
plt.show()

In [ ]:
plt.plot(angles, np.abs(np.array(global_fci_list)-np.array(global_dft_list)), label='error global DFT', color='b')
plt.plot(angles, np.abs(np.array(global_fci_list)-np.array(huz_H_diag_list)) , label='error huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(angles, np.abs(np.array(global_fci_list)-np.array(mu_H_diag_list)) , label='error mu emb', linestyle='-', marker='x', color='r')

plt.xlabel('angle (degrees)')
plt.ylabel('energy ERROR (Ha)')
plt.grid(True)
# plt.savefig("test.png")
plt.legend()
plt.show()

In [ ]:
ending = 'bottom_bottom_top_top_order.xyz'

bottom_bottom_top_top_file_paths = []
for xyz_filename in sorted(os.listdir(structure_dir)):
    if  xyz_filename.endswith(ending):
        bottom_bottom_top_top_file_paths.append(os.path.join(structure_dir, xyz_filename))

In [ ]:
# active part is bottom left 

with open(bottom_bottom_top_top_file_paths[0], 'r') as infile:
    xyz1 = infile.read()
with open(bottom_bottom_top_top_file_paths[int(len(bottom_bottom_top_top_file_paths)/2)], 'r') as infile:
    xyz2 = infile.read()
with open(bottom_bottom_top_top_file_paths[-1], 'r') as infile:
    xyz3 = infile.read()

numpy_xyz1_new = coords_as_numpy_array(xyz1)
numpy_xyz2_new = coords_as_numpy_array(xyz2)
numpy_xyz3_new = coords_as_numpy_array(xyz3)

print(bottom_bottom_top_top_file_paths[0][-42:])
print(bottom_bottom_top_top_file_paths[int(len(bottom_top_bottom_top_file_paths)/2)][-42:])
print(bottom_bottom_top_top_file_paths[-1][-42:])



plt.scatter(numpy_xyz1_new[:n_active_atoms,0], numpy_xyz1_new[:n_active_atoms,1], label='first step (85 degrees) active', marker='x', color='b')
plt.scatter(numpy_xyz1_new[n_active_atoms:,0], numpy_xyz1_new[n_active_atoms:,1], label='first step (85 degrees)', marker='s', color='b')

plt.scatter(numpy_xyz2_new[:n_active_atoms,0], numpy_xyz2_new[:n_active_atoms,1], label='middle step (90 degrees) active', marker='x', color='g')
plt.scatter(numpy_xyz2_new[n_active_atoms:,0], numpy_xyz2_new[n_active_atoms:,1], label='middle step (90 degrees)', marker='d', color='g')

plt.scatter(numpy_xyz3_new[:n_active_atoms,0], numpy_xyz3_new[:n_active_atoms,1], label='last step (95 degrees active)', marker='_', color='r', linewidths=4)
plt.scatter(numpy_xyz3_new[n_active_atoms:,0], numpy_xyz3_new[n_active_atoms:,1], label='last step (95 degrees)', marker='^', color='r')

plt.legend()
plt.show()

In [ ]:
numpy_xyz3_new

In [ ]:
list(zip(numpy_xyz3_new[n_active_atoms:,0], numpy_xyz3_new[n_active_atoms:,1]))

In [ ]:
(numpy_xyz3[:n_active_atoms,0], numpy_xyz3[:n_active_atoms,1])

# list(zip(numpy_xyz3[n_active_atoms:, 0], numpy_xyz3[n_active_atoms:, 1]))

In [ ]:
global_dft_list=[]
global_fci_list=[]
global_H_diag_list=[]
global_qubit_H_length_list=[]
global_qubit_H_nqubits=[]

huz_rhf_list=[]
huz_ccsd_list=[]
huz_fci_list=[]
huz_H_diag_list=[]
huz_qubit_H_length_list=[]
huz_H_nqubits=[]

mu_rhf_list=[]
mu_ccsd_list=[]
mu_fci_list=[]
mu_H_diag_list=[]
mu_qubit_H_length_list=[]
mu_H_nqubits=[]
N_active_AOs = []

for geometry_path in tqdm(bottom_bottom_top_top_file_paths):
    try:
        driver = NbedDriver(
                geometry=geometry_path,
                n_active_atoms=n_active_atoms,
                basis=basis,
                xc_functional=xc_functional,
                projector=projector,
                localization=localization,
                convergence=convergence,
                charge=charge,
                mu_level_shift=mu_level_shift,
                run_ccsd_emb=run_ccsd_emb,
                run_fci_emb=run_fci_emb,
                max_ram_memory=max_ram_memory,
                pyscf_print_level=pyscf_print_level,
                unit=unit,
                occupied_threshold=occupied_threshold,
                virtual_threshold=virtual_threshold)
    except:
        print('error for:')
        print(geometry_path)
        print()
        continue

    
    ## global
    global_dft_list.append(driver._global_rks.e_tot)
    global_fci_list.append(driver._global_fci.e_tot)
    
    global_H = driver.full_system_hamiltonian
    global_H_sparse = get_sparse_operator(global_H)
    E_fci =  sp.sparse.linalg.eigsh(global_H_sparse, k=1, which='SA')[0][0] + driver._global_hf.energy_nuc()
    global_H_diag_list.append(E_fci)
    
    global_qubit_Ham = getattr( HamiltonianConverter(global_H, transform='jordan_wigner'), 'openfermion')
    global_qubit_H_length_list.append(len(global_qubit_Ham.terms))
    global_qubit_H_nqubits.append(count_qubits(global_qubit_Ham))

    ##
    N_active_AOs.append(len(driver.localized_system.active_MO_inds))
    
    # HUZ
    huz_rhf_list.append(driver._huzinaga['e_rhf'])
    huz_ccsd_list.append(driver._huzinaga['e_ccsd'])
    huz_fci_list.append(driver._huzinaga['e_fci'])

    huz_H_emb = driver._huzinaga['hamiltonian']
    huz_classical = driver._huzinaga['classical_energy']
    huz_H_sparse = get_sparse_operator(huz_H_emb)
    huz_e_emb = sp.sparse.linalg.eigsh(huz_H_sparse, k=1, which='SA')[0][0]
    huz_H_diag_list.append(huz_e_emb)
    
    huz_qubit_Ham = getattr( HamiltonianConverter(huz_H_emb, transform='jordan_wigner'), 'openfermion')
    huz_qubit_H_length_list.append(len(huz_qubit_Ham.terms))
    huz_nqubits = count_qubits(huz_qubit_Ham)
    huz_H_nqubits.append(huz_nqubits)

    # mu shift
    mu_rhf_list.append(driver._mu['e_rhf'])
    mu_ccsd_list.append(driver._mu['e_ccsd'])
    mu_fci_list.append(driver._mu['e_fci'])
    
    mu_H_emb = driver._mu['hamiltonian']
    mu_classical = driver._mu['classical_energy']
    mu_H_sparse = get_sparse_operator(mu_H_emb)
    mu_e_emb = sp.sparse.linalg.eigsh(mu_H_sparse, k=1, which='SA')[0][0]
    mu_H_diag_list.append(mu_e_emb)
    
    mu_qubit_Ham = getattr( HamiltonianConverter(mu_H_emb, transform='jordan_wigner'), 'openfermion')
    mu_nqubits = count_qubits(huz_qubit_Ham)
    mu_qubit_H_length_list.append(len(mu_qubit_Ham.terms))
    mu_H_nqubits.append(mu_nqubits)
    
    del driver

In [ ]:
exp_data = {
    'global DFT': global_dft_list, 
    
    'global FCI': global_fci_list,
    'huz emb FCI': huz_H_diag_list,
    'mu emb FCI': mu_H_diag_list,
    
    'global H': global_qubit_H_length_list,
    'huz H emb': huz_qubit_H_length_list,
    'mu H emb': mu_qubit_H_length_list,
    
    'global n qubits ': global_qubit_H_nqubits,
    'huz n qubits ': huz_H_nqubits,
    'mu n qubits ': mu_H_nqubits,
    'N active AOs ': N_active_AOs,
    
}

df = pd.DataFrame(data=exp_data)

df

In [ ]:
%matplotlib inline

angles = list(range(85,96))
# angles = list(range(85,87))

plt.plot(angles, global_dft_list, label='global DFT', color='b')
plt.plot(angles, global_fci_list, label='global FCI', color='orange')
plt.plot(angles, huz_H_diag_list, label='huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(angles, mu_H_diag_list, label='mu emb', linestyle='-', marker='x', color='r')

plt.xlabel('angle (degrees)')
plt.ylabel('energy (Ha)')
plt.grid(True)
# plt.savefig("test.png")
plt.legend()
plt.show()

In [ ]:
plt.plot(angles, np.abs(np.array(global_fci_list)-np.array(global_dft_list)), label='error global DFT', color='b')
plt.plot(angles, np.abs(np.array(global_fci_list)-np.array(huz_H_diag_list)) , label='error huz emb', linestyle='dashed', marker='o', color='g')
plt.plot(angles, np.abs(np.array(global_fci_list)-np.array(mu_H_diag_list)) , label='error mu emb', linestyle='-', marker='x', color='r')

plt.xlabel('angle (degrees)')
plt.ylabel('energy ERROR (Ha)')
plt.grid(True)
# plt.savefig("test.png")
plt.legend()
plt.show()

In [ ]:
print(xyz)

In [ ]:
xyz[:xyz.find('\n')]

In [ ]:
np.
for line in xyz[4:].split('\n'):
    xyz_list = [float(xyz) for xyz in line.split('\t')[1:]]
    print(xyz_list)